## Using meta-analysis tools

Can we express quantitatively what we observed in the previous plots? This is where the meta-analysis tools come in: we used the R package [metafor](http://www.metafor-project.org/doku.php) to fit a mixed-effect (ME) model to the data reported for each measure. In this way, we can estimate an overall interval of R<sup>2</sup> values based on the effect sizes and the sample sizes. We can also estimate the interval of R<sup>2</sup> that we can expect in future studies (this is called prediction interval). A compact way to represent these results is given by forest plots: for each study, we represent the effect size and the related sample size using a square and a horizontal error bar; then for each measure, we represent the results from the ME model using a diamond and an additional error bar; finally to represent the prediction interval we use two hourglasses and a dotted line.

In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import plotly.colors
from plotly.subplots import make_subplots

from rpy2.robjects.packages import importr
import rpy2.robjects


In [ ]:
filtered_df = pd.read_pickle("filtered_df.pkl")

filtered_df['Variance'] = (4*filtered_df['R^2'])*((1-filtered_df['R^2'])**2)/filtered_df['Sample points']

measure_type = {'Diffusion':['RD', 'AD', 'FA', 'MD',
                'AWF', 'RK', 'RDe', 'MK'],
                'Magnetization transfer':['MTR',
                'ihMTR', 'MTR-UTE', 'MPF', 'MVF-MT',
                'R1f', 'T2m', 'T2f', 'k_mf','k_fm'],
                'T1 relaxometry':['T1'], 'T2 relaxometry':['T2', 'MWF', 'MVF-T2'],
                'Other':['QSM', 'R2*', 'rSPF', 'MTV',
                'T1p', 'T2p', 'RAFF', 'PD', 'T1sat']}

color_dict = {m:plotly.colors.qualitative.Bold[n]
              for n,m in enumerate(measure_type.keys())}

metafor = importr('metafor')
stats = importr('stats')

metastudy = {}
for m in filtered_df.Measure.unique():
    nstudies=len(filtered_df.Measure[filtered_df.Measure==m])
    if nstudies > 2:
        df_m = filtered_df[filtered_df.Measure==m]
        df_m = df_m.sort_values(by=['Year'])
        
        r2 = rpy2.robjects.FloatVector(df_m['R^2'])
        var = rpy2.robjects.FloatVector(df_m['Variance'])
        fit = metafor.rma(r2, var, method="REML", test="knha")
        res = stats.predict(fit)
        
        results = dict(zip(res.names,list(res)))
        
        metastudy[m] = dict(pred=results['pred'][0], cilb=results['pred'][0]-results['ci.lb'][0],
                            ciub=results['ci.ub'][0]-results['pred'][0],
                            crub=results['cr.ub'][0], 
                            crlb=results['cr.lb'][0])
measure_type_reverse={m:t for t,mlist in measure_type.items() for m in mlist}
fig5 = make_subplots(rows=3, cols=3, start_cell="top-left", vertical_spacing=0.05,
                      horizontal_spacing=0.2, x_title='R<sup>2</sup>',
                      subplot_titles=sorted(metastudy.keys(), key=measure_type_reverse.get))

row=1
col=1
for m in sorted(metastudy.keys(), key=measure_type_reverse.get):
    fig5.add_trace(go.Scatter(
        x=[round(metastudy[m]['crlb'],2) if round(metastudy[m]['crlb'],2)>0 else 0,
           round(metastudy[m]['crub'],2) if round(metastudy[m]['crub'],2)<1 else 1],
        y=['Mixed model','Mixed model'],
        line=dict(color='black', width=2, dash='dot'),
        hovertemplate = 'Prediction boundary: %{x}<extra></extra>',
        marker_symbol = 'hourglass-open', marker_size = 8
    ), row=row, col=col)
    
    fig5.add_trace(go.Scatter(
        x=[round(metastudy[m]['pred'],2)],
        y=['Mixed model'],
        mode='markers',
        marker = dict(color = 'black'),
        marker_symbol = 'diamond-wide',
        marker_size = 10,
        hovertemplate = 'R<sup>2</sup> estimate: %{x}<extra></extra>',
        error_x=dict(
            type='data',
            arrayminus=[round(metastudy[m]['cilb'],2) if round(metastudy[m]['cilb'],2)>0 else 0],
            array=[round(metastudy[m]['ciub'],2) if round(metastudy[m]['ciub'],2)<1 else 1])
    ), row=row, col=col)
    
    df_m = filtered_df[filtered_df.Measure==m]
    df_m = df_m.sort_values(by=['Year'], ascending=False)
    fig5.add_trace(go.Scatter(
        x=df_m['R^2'],
        y=df_m['Study'],
        text=df_m['Sample points'],
        mode='markers',
        marker = dict(color = color_dict[measure_type_reverse[m]]),
        marker_symbol = 'square',
        marker_size = np.log(50/df_m['Variance']),
        hovertemplate = '%{y}<br>R<sup>2</sup>: %{x}<br>Number of samples: %{text}<extra></extra>',
        error_x=dict(
            type='data',
            array=2*np.sqrt(df_m['Variance']))
    ), row=row, col=col)

    if col == 3:
        col = 1
        row += 1
    else:
        col += 1

fig5.update_xaxes(range=[0, 1])

fig5.update_layout(showlegend=False,
    title=dict(text='Figure 5: Forest plots and mixed modelling results',x=0.1),
    margin=dict(l=200),
    width=1000,
    height=1400)
fig5.show()                        

Can some of this variance be explained by the differences in methodological choices and experimental conditions we mention? The number of studies is limited for a quantitative evaluation, but we can get a qualitative idea using bar plots and scatter plots organized by each condition.

In [ ]:
structures={'Lesions':'Lesions',
            'Substantia nigra':'Deep grey matter',
            'Hippocampal commissure':'White matter',
            'Putamen':'Deep grey matter',
            'Motor cortex':'Grey matter',
            'Globus pallidus':'Deep grey matter',
            'Perforant pathway':'White matter',
            'Mammilothalamic tract':'White matter',
            'External capsule':'White matter',
            'Inter-peduncular nuclues':'Deep grey matter',
            'Hippocampus':'Deep grey matter',
            'Thalamic nuclei':'Deep grey matter',
            'Thalamus':'Deep grey matter',
            'Cerebellum':'Grey matter',
            'Amygdala':'Deep grey matter',
            'Cingulum':'White matter',
            'Striatum':'Deep grey matter',
            'Accumbens':'Deep grey matter',
            'Basal ganglia':'Deep grey matter',
            'Anterior commissure':'White matter',
            'Cortex':'Grey matter',
            'Fimbria':'White matter',
            'Somatosensory cortex':'Grey matter',
            'Dorsal tegmental tract':'White matter',
            'Superior colliculus':'Deep grey matter',
            'Fasciculus retroflexus':'White matter',
            'Optic nerve':'White matter',
            'Dentate gyrus':'Grey matter',
            'Corpus callosum':'White matter',
            'Fornix':'White matter',
            'White matter':'White matter',
            'Grey matter':'Grey matter',
            'Optic tract':'White matter',
            'Internal capsule':'White matter',
            'Stria medullaris':'White matter'}


tissue_types=[]
for s in filtered_df['Specific structure(s)']:
    t_list=[]
    for i in s.split(','):
        t_list.append(structures[i.strip()])
    tissue_types.append('+'.join(list(set(t_list))))

filtered_df['Tissue types']=tissue_types

fig6 = make_subplots(rows=3, cols=1, start_cell="top-left", vertical_spacing=0.2, y_title='R<sup>2</sup>',
                     subplot_titles=['R<sup>2</sup> values and reference techniques',
                                     'R<sup>2</sup> values and pathology',
                                     'R<sup>2</sup> values and tissue types'])

references = ['Histology', 'Immunohistochemistry', 'Microscopy', 'EM']

for r in references:
    df_r=filtered_df[filtered_df['Histology/microscopy measure'].str.contains(r)]
    fig6.add_trace(go.Box(
        y=df_r['R^2'],
        x=df_r['Histology/microscopy measure'],
        boxpoints='all',
        text=df_r['Measure'] + ' - ' + df_r['Study'],
        name=r
    ), col=1, row=1)

for t in filtered_df['Condition'].unique():
    df_t=filtered_df[filtered_df['Condition']==t]
    fig6.add_trace(go.Box(
        y=df_t['R^2'],
        x=df_t['Condition'],
        boxpoints='all',
        text=df_t['Measure'] + ' - ' + df_t['Study'],
        name=t
    ), col=1, row=2)

for t in filtered_df['Tissue types'].unique():
    df_t=filtered_df[filtered_df['Tissue types']==t]
    fig6.add_trace(go.Box(
        y=df_t['R^2'],
        x=df_t['Tissue types'],
        boxpoints='all',
        text=df_t['Measure'] + ' - ' + df_t['Study'],
        name=t
    ), col=1, row=3)   

fig6.update_layout(
    title=dict(
        text='Figure 6: Experimental conditions and methodological choices influencing the R<sup>2</sup> values',
        x=0.1),
    margin=dict(l=100),
    showlegend=False,
    height=1200,
    width=900
)

fig6.show()

In [ ]:
fig7 = make_subplots(rows=2, cols=2, start_cell="top-left", vertical_spacing=0.15, y_title='R<sup>2</sup>',
                     subplot_titles=['R<sup>2</sup> values and magnetic fields',
                                     'R<sup>2</sup> values and tissue conditions',
                                     'R<sup>2</sup> values and co-registration',
                                     'R<sup>2</sup> values and human/animal tissue'
                                    ])

for m in measure_type.keys():
    df_m=filtered_df[filtered_df["Measure"].isin(measure_type[m])]
    fig7.add_trace(go.Scatter(x=df_m['Magnetic field'],
                              y=df_m['R^2'],
                              text=df_m['Measure'] + ' - ' + df_m['Study'],
                              marker=dict(color=color_dict[m]),
                              name=m,
                              mode='markers'), col=1, row=1)

fig7.update_layout(
    xaxis=dict(title='Magnetic field [T]')
)

for t in filtered_df['Tissue condition'].unique():
    df_t=filtered_df[filtered_df['Tissue condition']==t]
    fig7.add_trace(go.Box(
        y=df_t['R^2'],
        x=df_t['Tissue condition'],
        boxpoints='all',
        text=df_t['Measure'] + ' - ' + df_t['Study'],
        name=t
    ), col=2, row=1)
    
for t in filtered_df['Co-registration'].unique():
    df_t=filtered_df[filtered_df['Co-registration']==t]
    fig7.add_trace(go.Box(
        y=df_t['R^2'],
        x=df_t['Co-registration'],
        boxpoints='all',
        text=df_t['Measure'] + ' - ' + df_t['Study'],
        name=t
    ), col=1, row=2)
    
for t in filtered_df['Human/animal'].unique():
    df_t=filtered_df[filtered_df['Human/animal']==t]
    fig7.add_trace(go.Box(
        y=df_t['R^2'],
        x=df_t['Human/animal'],
        boxpoints='all',
        text=df_t['Measure'] + ' - ' + df_t['Study'],
        name=t
    ), col=2, row=2)

fig7.update_layout(
    title=dict(text='Figure 7: Other factors to consider when assessing R<sup>2</sup>', x=0.1),
    margin=dict(l=100),
    showlegend=False,
    height=900,
    width=900
)    
    
fig7.show()